In [1]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Configura el navegador con undetected_chromedriver y un User-Agent aleatorio
options = uc.ChromeOptions()
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
]
options.add_argument(f"user-agent={random.choice(user_agents)}")
browser = uc.Chrome(options=options)

# URL inicial: Página de listados de Madrid
base_url = "https://www.pisos.com/alquiler/pisos-madrid/"
browser.get(base_url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception:
    print("No se encontró el botón de cookies o ya fue aceptado.")
time.sleep(5)  # Añadir pausa después de aceptar cookies

# Lista para almacenar los datos
all_data = []
inmueble_counter = 1  # Contador de inmuebles
max_inmuebles = 2600  # Número máximo de inmuebles que queremos recolectar

# Función para extraer datos de un inmueble dado su enlace
def extraer_datos_inmueble():
    try:
        WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')

        # Extraer datos del inmueble
        try:
            # Descripción del inmueble (H1)
            descripcion = soup_inmueble.find('h1').text.strip() if soup_inmueble.find('h1') else "Descripción no disponible"

            # Localización en el primer <p> después del <h1> (ejemplo: Centro (Galapagar))
            localizacion = soup_inmueble.find('p').text.strip() if soup_inmueble.find('p') else "Localización no disponible"

            # Precio
            precio_element = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'})
            precio = precio_element.text.split(' ')[0] if precio_element else "N/A"

            # Superficie construida
            superficie_element = soup_inmueble.find('span', {'class': 'features__value'})
            superficie_construida = superficie_element.text.split(' ')[0] if superficie_element else "N/A"

            # Última actualización
            ultima_actualizacion_element = soup_inmueble.find('p', {'class': 'last-update__date'})
            ultima_actualizacion = ultima_actualizacion_element.text if ultima_actualizacion_element else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'})
                    value = feature.find('span', {'class': 'features__value'})
                    if label and value:
                        features_list.append((label.get_text(strip=True), value.get_text(strip=True)))
            else:
                features_list = "N/A"

            # Certificado energético
            energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
            if energy_certificate:
                consumo_element = energy_certificate.find('div', {'class': 'energy-certificate__data'})
                consumo = consumo_element.find_all('span')[1].get_text(strip=True) if consumo_element else "N/A"
                
                emisiones_element = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})
                emisiones = emisiones_element[1].find_all('span')[1].get_text(strip=True) if len(emisiones_element) > 1 else "N/A"
            else:
                consumo = "N/A"
                emisiones = "N/A"

            # Guarda los datos en una lista
            data = {
                'Descripción': descripcion,
                'Localización': localizacion,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list,
                'Tipo de operación': 'Alquiler'
            }

            return data

        except Exception as e:
            print(f"Error al extraer datos: {e}")
            return None

    except Exception as e:
        print(f"Error al cargar la página del inmueble: {e}")
        return None

# Función para avanzar al siguiente inmueble
def ir_al_siguiente_inmueble():
    try:
        siguiente_boton = WebDriverWait(browser, 20).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.navigation__link.navigation__link--next'))
        )
        siguiente_boton.click()
        time.sleep(2)  # Pausa para cargar el siguiente inmueble
        return True
    except Exception as e:
        print(f"No se encontró el botón de 'Siguiente': {e}")
        return False

# Iniciar en el primer inmueble
try:
    primer_inmueble = WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.ad-preview'))
    )
    primer_inmueble.click()
    time.sleep(2)  # Pausa para cargar la página del primer inmueble
except Exception as e:
    print(f"Error al abrir el primer inmueble: {e}")

# Bucle para extraer datos de cada inmueble hasta alcanzar el máximo o agotar los inmuebles
while len(all_data) < max_inmuebles:
    data = extraer_datos_inmueble()
    if data:
        all_data.append(data)
        print(f"Inmueble {inmueble_counter}: {data}")
        inmueble_counter += 1

    # Intentar ir al siguiente inmueble
    if not ir_al_siguiente_inmueble():
        break  # Si no hay más inmuebles, salir del bucle

# Convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Opcional: guarda el DataFrame en un archivo CSV
df.to_csv('alquiler_inmuebles.csv', index=False)

# Cierra el navegador
browser.quit()


Inmueble 1: {'Descripción': 'Casa unifamiliar en alquiler en Calle XIX', 'Localización': 'Las Matas-Los Peñascales (Las Rozas de Madrid)', 'Precio': '4.000', 'Superficie Construida': '600', 'Última Actualización': 'Anuncio actualizado el 28/10/2024', 'Consumo Energético': 'N/A', 'Emisiones CO2': 'N/A', 'Características': [('Superficie construida:', '600 m²'), ('Superficie útil:', '600 m²'), ('Habitaciones:', '6'), ('Baños:', '7'), ('Tipo de casa:', 'Unifamiliar'), ('Antigüedad:', 'Entre 5 y 10 años'), ('Conservación:', 'En buen estado'), ('Referencia:', '2805-002317')], 'Tipo de operación': 'Alquiler'}
Inmueble 2: {'Descripción': 'Piso en alquiler en Zona Avenida de Europa', 'Localización': 'Zona Avenida de Europa (Pozuelo de Alarcón)', 'Precio': '1.400', 'Superficie Construida': '65', 'Última Actualización': 'Anuncio actualizado el 28/10/2024', 'Consumo Energético': 'N/A', 'Emisiones CO2': 'N/A', 'Características': [('Superficie construida:', '65 m²'), ('Habitaciones:', '1'), ('Baños:

In [2]:
df

,Descripción,Localización,Precio,Superficie Construida,Última Actualización,Consumo Energético,Emisiones CO2,Características,Tipo de operación
0,Casa unifamiliar en alquiler en Calle XIX,Las Matas-Los Peñascales (Las Rozas de Madrid),4.000,600,Anuncio actualizado el 28/10/2024,N/A,N/A,"[(Superficie construida:, 600 m²), (Superficie...",Alquiler
1,Piso en alquiler en Zona Avenida de Europa,Zona Avenida de Europa (Pozuelo de Alarcón),1.400,65,Anuncio actualizado el 28/10/2024,N/A,N/A,"[(Superficie construida:, 65 m²), (Habitacione...",Alquiler
2,Piso en alquiler en Calle Luis Garcia Cereceda...,La Finca (Pozuelo de Alarcón),5.500,190,Anuncio actualizado el 28/10/2024,Consumo:15 kWh/m² año,Emisiones:2 Kg CO₂/m² año,"[(Superficie construida:, 190 m²), (Superficie...",Alquiler
3,"Estudio en alquiler en Calle de Pollensa, 5",Parque Empresarial (Las Rozas de Madrid),1.147,50,Anuncio actualizado el 03/07/2024,N/A,N/A,"[(Superficie construida:, 50 m²), (Baños:, 1),...",Alquiler
4,"Piso en alquiler en Calle Gran Vía, cerca de C...",Universidad-Malasaña (Distrito Centro. Madrid ...,3.500,122,Anuncio actualizado el 03/10/2024,Consumo:70 kWh/m² año,Emisiones:15 Kg CO₂/m² año,"[(Superficie construida:, 122 m²), (Superficie...",Alquiler
...,...,...,...,...,...,...,...,...,...
2114,Piso en alquiler en Calle de la Maquinilla,¡Vaya! Esta propiedad no tiene fotos,950,45,Anuncio actualizado el 28/10/2024,Consumo:30 kWh/m² año,Emisiones:179 Kg CO₂/m² año,"[(Superficie construida:, 45 m²), (Habitacione...",Alquiler
2115,"Dúplex en alquiler en Avenida de Valdemarín, c...",Valdemarín (Distrito Moncloa-Aravaca. Madrid C...,3.900,220,Anuncio actualizado el 28/10/2024,N/A,N/A,"[(Superficie construida:, 220 m²), (Superficie...",Alquiler
2116,Piso en alquiler en Cortes-Huertas,Cortes-Huertas (Distrito Centro. Madrid Capital),1.200,52,Anuncio actualizado el 28/10/2024,Consumo:190 kWh/m² año,Emisiones:32 Kg CO₂/m² año,"[(Superficie construida:, 52 m²), (Habitacione...",Alquiler
2117,Piso en alquiler en Calle de los Misterios,Quintana (Distrito Ciudad Lineal. Madrid Capital),950,38,Anuncio actualizado el 28/10/2024,N/A,N/A,"[(Superficie construida:, 38 m²), (Superficie ...",Alquiler


In [3]:
cantidad_unicos = df["Descripción"].nunique()
print(cantidad_unicos)


1381


In [4]:
cantidad_unicos = df["Precio"].nunique()
print(cantidad_unicos)


474


In [6]:
cantidad_unicos = df["Localización"].nunique()
print(cantidad_unicos)


307


In [8]:
# Verificar duplicados en función de descripción y localización
inmuebles_duplicados = df[df.duplicated(subset=['Descripción', 'Localización'], keep=False)]

# Contar el número de inmuebles duplicados que tienen la misma descripción y localización
num_duplicados = inmuebles_duplicados.shape[0]
print(f"Inmuebles duplicados por Descripción y Localización: {num_duplicados}")

# Verificar si los duplicados también tienen el mismo precio
duplicados_mismo_precio = inmuebles_duplicados[
    inmuebles_duplicados.duplicated(subset=['Descripción', 'Localización', 'Precio'], keep=False)
]

# Contar el número de inmuebles duplicados que tienen la misma descripción, localización y precio
num_duplicados_mismo_precio = duplicados_mismo_precio.shape[0]
print(f"Inmuebles duplicados por Descripción, Localización y Precio: {num_duplicados_mismo_precio}")

# Convertir la columna 'Características' a cadena de texto para poder buscar duplicados
duplicados_mismo_precio['Características'] = duplicados_mismo_precio['Características'].apply(lambda x: str(x))

# Verificar si los duplicados también tienen las mismas características
duplicados_mismo_precio_caracteristicas = duplicados_mismo_precio[
    duplicados_mismo_precio.duplicated(subset=['Descripción', 'Localización', 'Precio', 'Características'], keep=False)
]

# Contar el número de inmuebles duplicados que tienen la misma descripción, localización, precio y características
num_duplicados_mismo_precio_caracteristicas = duplicados_mismo_precio_caracteristicas.shape[0]
print(f"Inmuebles duplicados por Descripción, Localización, Precio y Características: {num_duplicados_mismo_precio_caracteristicas}")

duplicados_mismo_precio_caracteristicas

Inmuebles duplicados por Descripción y Localización: 879
Inmuebles duplicados por Descripción, Localización y Precio: 102
Inmuebles duplicados por Descripción, Localización, Precio y Características: 2


C:\Users\guill\AppData\Local\Temp\ipykernel_24208\849320747.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicados_mismo_precio['Características'] = duplicados_mismo_precio['Características'].apply(lambda x: str(x))


,Descripción,Localización,Precio,Superficie Construida,Última Actualización,Consumo Energético,Emisiones CO2,Características,Tipo de operación
217,"Piso en alquiler en Calle Colombia, cerca de C...",Hispanoamérica (Distrito Chamartín. Madrid Cap...,1.500,68,Anuncio actualizado el 22/10/2024,N/A,N/A,"[('Superficie construida:', '68 m²'), ('Superf...",Alquiler
231,"Piso en alquiler en Calle Colombia, cerca de C...",Hispanoamérica (Distrito Chamartín. Madrid Cap...,1.500,68,Anuncio actualizado el 22/10/2024,N/A,N/A,"[('Superficie construida:', '68 m²'), ('Superf...",Alquiler
